### UniprotファイルからFT行にTRANSMEMがあるエントリを抽出する

In [ ]:
import sys

def extract_transmembrane_entries(input_file, output_file):
    total_entries = 0
    transmem_entries = 0
    with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
        entry_buffer = []
        is_transmem = False
        for line in f_in:
            entry_buffer.append(line)
            if line.startswith('FT') and 'TRANSMEM' in line:
                is_transmem = True
            
            if line.strip() == '//':
                total_entries += 1
                if is_transmem:
                    transmem_entries += 1
                    f_out.write("".join(entry_buffer))
                entry_buffer = []
                is_transmem = False
    return total_entries, transmem_entries

if __name__ == '__main__':
    input_filename = 'uniprot_sprot_human.dat'
    output_filename = 'uniprot_sprot_human_transmem.dat'
    total, transmem = extract_transmembrane_entries(input_filename, output_filename)
    print(f"全エントリ数: {total}")
    print(f"抽出された膜貫通タンパク質のエントリ数: {transmem}")
    print(f"抽出したエントリは {output_filename} に保存されました。") 

### UniprotファイルからACを1つずつ抽出する

In [ ]:
import sys

def extract_primary_accession_numbers(input_file):
    """
    Uniprotデータファイルから各エントリの最初の(プライマリ)アクセッション番号(AC)を抽出する。
    """
    accession_numbers = []
    with open(input_file, 'r') as f:
        ac_found_for_this_entry = False
        for line in f:
            # '//'はエントリの区切り
            if line.strip() == '//':
                ac_found_for_this_entry = False
                continue

            # AC行で、かつこのエントリでまだACを見つけていない場合
            if line.startswith('AC') and not ac_found_for_this_entry:
                try:
                    # AC   P04637; Q16254; -> P04637
                    ac = line.split()[1].rstrip(';')
                    accession_numbers.append(ac)
                    ac_found_for_this_entry = True
                except IndexError:
                    # AC行が空など、予期せぬフォーマットの場合はスキップ
                    pass
    return accession_numbers

if __name__ == '__main__':
    input_filename = 'uniprot_sprot_human_transmem.dat'
    output_filename = 'accession_transmem.txt'
    try:
        accessions = extract_primary_accession_numbers(input_filename)
        
        with open(output_filename, 'w') as f_out:
            for ac in accessions:
                f_out.write(ac + '\n')

        print(f"ファイル '{input_filename}' から {len(accessions)} 個のアクセッション番号を抽出し、'{output_filename}' に保存しました。")

    except FileNotFoundError:
        print(f"エラー: 入力ファイル '{input_filename}' が見つかりません。")
        sys.exit(1) 

### Gタンパク質共役型受容体を取り出す

In [ ]:
import sys

def extract_gpcr_accessions(input_file):
    """
    Uniprotデータファイルから、'G protein-coupled receptor' に関連する
    エントリのプライマリACを抽出する。

    条件:
    - DR行に 'GO;' が含まれる
    - DR行に 'G protein-coupled receptor' が含まれる
    """
    gpcr_acs = []
    
    with open(input_file, 'r') as f:
        current_entry_lines = []
        for line in f:
            current_entry_lines.append(line)
            
            # '//' はエントリの終わりを示す
            if line.strip() == '//':
                # エントリが条件を満たすかチェック
                is_gpcr = False
                primary_ac = None
                
                for entry_line in current_entry_lines:
                    # エントリの最初のAC番号を取得
                    if entry_line.startswith('AC') and primary_ac is None:
                        try:
                            primary_ac = entry_line.split()[1].rstrip(';')
                        except IndexError:
                            pass # 不正なAC行をスキップ
                    
                    # GPCRの条件をチェック
                    if (entry_line.startswith('DR') and 
                        'GO;' in entry_line and 
                        'G protein-coupled receptor' in entry_line):
                        is_gpcr = True

                # 条件を満たしていれば、ACをリストに追加
                if is_gpcr and primary_ac:
                    gpcr_acs.append(primary_ac)
                
                # 次のエントリのためにバッファをリセット
                current_entry_lines = []
                
    return gpcr_acs

if __name__ == '__main__':
    # 全てのタンパク質が含まれるファイルを入力とする
    input_filename = 'uniprot_sprot_human.dat'
    output_filename = 'ac_gpc_receptors.txt'
    
    try:
        accessions = extract_gpcr_accessions(input_filename)
        
        with open(output_filename, 'w') as f_out:
            for ac in accessions:
                f_out.write(ac + '\n')
                
        print(f"'{input_filename}' から {len(accessions)} 個の G protein-coupled receptor のACを抽出し、")
        print(f"'{output_filename}' に保存しました。")
        
    except FileNotFoundError:
        print(f"エラー: 入力ファイル '{input_filename}' が見つかりません。")
        sys.exit(1) 

### ミトコンドリア内膜タンパク質を抽出する

In [ ]:
import sys

def extract_mito_inner_membrane_accessions(input_file):
    """
    Uniprotデータファイルから、'mitochondrial inner membrane' に関連する
    エントリのプライマリACを抽出する。

    条件:
    - DR行に 'GO;' が含まれる
    - DR行に 'mitochondrial inner membrane' が含まれる
    """
    mito_acs = []
    
    with open(input_file, 'r') as f:
        current_entry_lines = []
        for line in f:
            current_entry_lines.append(line)
            
            # '//' はエントリの終わりを示す
            if line.strip() == '//':
                # エントリが条件を満たすかチェック
                is_mito_inner_membrane = False
                primary_ac = None
                
                for entry_line in current_entry_lines:
                    # エントリの最初のAC番号を取得
                    if entry_line.startswith('AC') and primary_ac is None:
                        try:
                            primary_ac = entry_line.split()[1].rstrip(';')
                        except IndexError:
                            pass # 不正なAC行をスキップ
                    
                    # mitochondrial inner membrane の条件をチェック
                    if (entry_line.startswith('DR') and 
                        'GO;' in entry_line and 
                        'mitochondrial inner membrane' in entry_line):
                        is_mito_inner_membrane = True

                # 条件を満たしていれば、ACをリストに追加
                if is_mito_inner_membrane and primary_ac:
                    mito_acs.append(primary_ac)
                
                # 次のエントリのためにバッファをリセット
                current_entry_lines = []
                
    return mito_acs

if __name__ == '__main__':
    # 全てのタンパク質が含まれるファイルを入力とする
    input_filename = 'uniprot_sprot_human.dat'
    output_filename = 'ac_mito_inner_membrane.txt'
    
    try:
        accessions = extract_mito_inner_membrane_accessions(input_filename)
        
        with open(output_filename, 'w') as f_out:
            for ac in accessions:
                f_out.write(ac + '\n')
                
        print(f"'{input_filename}' から {len(accessions)} 個の mitochondrial inner membrane のACを抽出し、")
        print(f"'{output_filename}' に保存しました。")
        
    except FileNotFoundError:
        print(f"エラー: 入力ファイル '{input_filename}' が見つかりません。")
        sys.exit(1) 

### 膜貫通回数ごとにまとめるスクリプト

In [ ]:
import sys
from collections import defaultdict

def classify_transmem_proteins(input_file):
    """
    膜貫通タンパク質をTRANSMEMの出現回数によって分類し、ACを返す。

    戻り値:
        dict: キーが貫通回数(1-12, '13+'), 値がACのリストである辞書
    """
    # defaultdictを使うと、キーが存在しない場合に自動で空のリストを作成してくれる
    classified_acs = defaultdict(list)
    
    with open(input_file, 'r') as f_in:
        current_ac = None
        transmem_count = 0
        
        for line in f_in:
            # エントリの最初のACを取得 (プライマリAC)
            if line.startswith('AC') and current_ac is None:
                try:
                    current_ac = line.split()[1].rstrip(';')
                except IndexError:
                    pass # 不正な形式のAC行は無視

            # TRANSMEMを持つFT行を数える
            if line.startswith('FT') and 'TRANSMEM' in line:
                transmem_count += 1
            
            # エントリの終わりで処理
            if line.strip() == '//':
                if current_ac and transmem_count > 0:
                    if 1 <= transmem_count <= 12:
                        category = transmem_count
                    else: # 13回以上
                        category = '13+'
                    
                    classified_acs[category].append(current_ac)

                # 次のエントリのためにリセット
                current_ac = None
                transmem_count = 0
                
    return classified_acs

if __name__ == '__main__':
    # 膜貫通タンパク質が既に抽出されたファイルを入力とする
    input_filename = 'uniprot_sprot_human_transmem.dat'
    
    try:
        results = classify_transmem_proteins(input_filename)
        
        print("膜貫通回数ごとのAC抽出結果:")
        print("="*30)

        # 1-12回と13+回を順番に処理
        categories = list(range(1, 13)) + ['13+']
        
        for category in categories:
            if category in results:
                ac_list = results[category]
                # '13+' をファイル名として使えるように '13plus' に置換
                filename_cat = str(category).replace('+', 'plus')
                output_filename = f"ac_transmem_{filename_cat}pass.txt"
                
                with open(output_filename, 'w') as f_out:
                    for ac in ac_list:
                        f_out.write(ac + '\n')
                
                print(f"カテゴリ '{category}回': {len(ac_list):>4}個のACを'{output_filename}'に保存しました。")
            else:
                 print(f"カテゴリ '{category}回': 該当なし")


    except FileNotFoundError:
        print(f"エラー: 入力ファイル '{input_filename}' が見つかりません。")
        print("まず 'extract_transmem_proteins.py' を実行して、膜貫通タンパク質ファイルを生成してください。")
        sys.exit(1) 

### signaling receptor activityを抽出する

In [ ]:
import sys

def extract_signaling_receptor_entries(input_file, output_file):
    """
    Uniprotデータファイルから 'signaling receptor activity' を持つエントリを
    まるごと抽出して新しいファイルに書き出す。

    条件:
    - DR行に 'GO;' が含まれる
    - DR行に 'signaling receptor activity' が含まれる
    """
    entries_found = 0
    
    with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
        entry_buffer = []
        is_target_entry = False
        
        for line in f_in:
            entry_buffer.append(line)
            
            # 条件をチェック
            if (line.startswith('DR') and 
                'GO;' in line and 
                'signaling receptor activity' in line):
                is_target_entry = True
            
            # エントリの終わりで処理
            if line.strip() == '//':
                if is_target_entry:
                    f_out.write("".join(entry_buffer))
                    entries_found += 1
                
                # 次のエントリのためにリセット
                entry_buffer = []
                is_target_entry = False
                
    return entries_found

if __name__ == '__main__':
    # 全てのタンパク質が含まれるファイルを入力とする
    input_filename = 'uniprot_sprot_human.dat'
    output_filename = 'uniprot_sprot_human_signaling_receptor.dat'
    
    try:
        count = extract_signaling_receptor_entries(input_filename, output_filename)
        
        print(f"'{input_filename}' から 'signaling receptor activity' を持つ {count} 個のエントリを抽出し、")
        print(f"'{output_filename}' に保存しました。")
        
    except FileNotFoundError:
        print(f"エラー: 入力ファイル '{input_filename}' が見つかりません。")
        sys.exit(1) 

### 膜貫通領域の位置と回数が分かるjsonの生成

In [ ]:
import sys
from collections import defaultdict

def classify_transmem_proteins(input_file):
    """
    膜貫通タンパク質をTRANSMEMの出現回数によって分類し、ACを返す。

    戻り値:
        dict: キーが貫通回数(1-12, '13+'), 値がACのリストである辞書
    """
    # defaultdictを使うと、キーが存在しない場合に自動で空のリストを作成してくれる
    classified_acs = defaultdict(list)
    
    with open(input_file, 'r') as f_in:
        current_ac = None
        transmem_count = 0
        
        for line in f_in:
            # エントリの最初のACを取得 (プライマリAC)
            if line.startswith('AC') and current_ac is None:
                try:
                    current_ac = line.split()[1].rstrip(';')
                except IndexError:
                    pass # 不正な形式のAC行は無視

            # TRANSMEMを持つFT行を数える
            if line.startswith('FT') and 'TRANSMEM' in line:
                transmem_count += 1
            
            # エントリの終わりで処理
            if line.strip() == '//':
                if current_ac and transmem_count > 0:
                    if 1 <= transmem_count <= 12:
                        category = transmem_count
                    else: # 13回以上
                        category = '13+'
                    
                    classified_acs[category].append(current_ac)

                # 次のエントリのためにリセット
                current_ac = None
                transmem_count = 0
                
    return classified_acs

if __name__ == '__main__':
    # 膜貫通タンパク質が既に抽出されたファイルを入力とする
    input_filename = 'uniprot_sprot_human_signaling_receptor.dat'
    
    try:
        results = classify_transmem_proteins(input_filename)
        
        print("膜貫通回数ごとのAC抽出結果:")
        print("="*30)

        # 1-12回と13+回を順番に処理
        categories = list(range(1, 13)) + ['13+']
        
        for category in categories:
            if category in results:
                ac_list = results[category]
                # '13+' をファイル名として使えるように '13plus' に置換
                filename_cat = str(category).replace('+', 'plus')
                output_filename = f"ac_transmem_{filename_cat}pass_signaling_receptor.txt"
                
                with open(output_filename, 'w') as f_out:
                    for ac in ac_list:
                        f_out.write(ac + '\n')
                
                # print(f"カテゴリ '{category}回': {len(ac_list):>4}個のACを'{output_filename}'に保存しました。")
                print(f"{category}回\t{len(ac_list):>4}")
            else:
                 print(f"カテゴリ '{category}回': 該当なし")


    except FileNotFoundError:
        print(f"エラー: 入力ファイル '{input_filename}' が見つかりません。")
        print("まず 'extract_transmem_proteins.py' を実行して、膜貫通タンパク質ファイルを生成してください。")
        sys.exit(1) 